In [1]:
# 1d Close < BB_M, Green Vol > 前根 Red Vol, rsi >= 42, 出場: k & d >= 80, Close <= BB_U, rsi >= 68

In [2]:
import requests
import json
import pandas as pd
import datetime as dt
import time
import ta
import os

In [3]:
# 調整 notebook 寬物
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% ! important; }<style>"))

# 調整 output 不折疊
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [4]:
import winsound
frequency = 250  # Set Frequency To 2500 Hertz
duration = 300  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [5]:
from binance.client import Client
from dotenv import load_dotenv

# Get the path to the current directory
current_directory = os.getcwd()

# Specify the path to the .env file relative to the current directory
dotenv_path = os.path.join(current_directory, '.env')

# Load the environment variables from the .env file
load_dotenv(dotenv_path)

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret, testnet = False)

spot = client.get_account()
s_bal = pd.DataFrame(spot['balances'])
print(s_bal)

     asset        free      locked
0      BTC  0.00000000  0.00000000
1      LTC  0.00000000  0.00000000
2      ETH  0.00000000  0.00000000
3      NEO  0.00000000  0.00000000
4      BNB  0.04622450  0.00000000
..     ...         ...         ...
536    EDU  0.00000000  0.00000000
537  WBETH  0.00000000  0.00000000
538    SUI  0.00000000  0.00000000
539   PEPE        0.00        0.00
540  FLOKI        0.00        0.00

[541 rows x 3 columns]


In [6]:
timezone = 8
symbol = 'ethusdt'
interval = '1d'

# start epoch till now, use prior 10 days for this strategy
start_time = round(time.time() * 1000 - 86400 * 1000 * 40)
pre_start = pd.to_datetime(start_time, unit='ms') + pd.Timedelta(hours=timezone)
pre_start = pre_start.strftime('%Y-%m-%d %H:%M:%S')
print(f'Start time: {pre_start}')

end_time = round(time.time() * 1000)

# step between timestamps in milliseconds, 60000 = 1min 
step = 60000 * 3600 * 60

Start time: 2023-04-12 23:25:46


In [7]:
# 先抓歷史資料好讓技術指標能成型
def get_historical(symbol, interval, start_time, end_time, step):
    
    raw_df = pd.DataFrame()
    
    url = "https://api.binance.com/api/v3/klines"
    
    for timestamp in range(start_time, end_time, step):
        params = {"symbol": symbol.upper(),
                  "interval": interval,
                  "startTime": timestamp,
                  "endTime": timestamp + step}
        response = requests.get(url, params=params).json()
        out = pd.DataFrame(response, columns = ["Open time", "Open", "High", "Low", "Close",
                                               "Volume", "Close_Time", "Quote asset volume",
                                               "Number of trades", "Taker buy base asset volume",
                                               "Taker buy quote asset volume", "Ignore"])
        raw_df = pd.concat([raw_df, out], axis = 0)
    
    raw_df = raw_df[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']]
    convert_dict = {'Close_Time': float, 'Open': float, 'Close': float, "High": float, "Low": float, 'Volume': float}
    raw_df = raw_df.astype(convert_dict)

    raw_df['Close_Time'] = pd.to_datetime(raw_df['Close_Time'], unit = 'ms')
    raw_df['Close_Time'] = raw_df['Close_Time'] + pd.Timedelta(hours=timezone)
    raw_df['Close_Time'] = raw_df['Close_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    raw_df = raw_df.reset_index(drop=True)
    
    return raw_df

In [8]:
raw_df = get_historical(symbol, interval, start_time, end_time, step)
print(raw_df)

             Close_Time     Open    Close     High      Low       Volume
0   2023-04-14 07:59:59  1917.40  2012.01  2023.00  1899.55  676164.5188
1   2023-04-15 07:59:59  2012.00  2099.99  2128.76  2009.22  896307.3934
2   2023-04-16 07:59:59  2099.99  2090.60  2111.85  2071.13  321823.7851
3   2023-04-17 07:59:59  2090.61  2118.67  2141.54  2072.72  319880.4393
4   2023-04-18 07:59:59  2118.66  2074.00  2120.51  2056.25  426972.7071
5   2023-04-19 07:59:59  2073.99  2103.50  2125.00  2051.00  414244.6135
6   2023-04-20 07:59:59  2103.51  1933.73  2104.60  1923.03  775389.0602
7   2023-04-21 07:59:59  1933.74  1942.98  1982.99  1914.43  625336.2495
8   2023-04-22 07:59:59  1942.98  1848.78  1957.27  1825.00  688876.6186
9   2023-04-23 07:59:59  1848.77  1874.16  1887.31  1841.64  331007.5879
10  2023-04-24 07:59:59  1874.17  1862.00  1882.01  1836.37  344514.3444
11  2023-04-25 07:59:59  1862.01  1841.84  1890.00  1806.00  447864.0517
12  2023-04-26 07:59:59  1841.84  1866.10  1877.86 

In [9]:
def indicators(df):

# bband
    bb_int = 30
    bb_dev = 2
    bb = ta.volatility.BollingerBands(df['Close'], window=bb_int, window_dev=bb_dev)
    df['bb_u'] = bb.bollinger_hband()
    df['bb_m'] = bb.bollinger_mavg()
    df['bb_l'] = bb.bollinger_lband()

# kd
    length = 14
    smooth = 3
    df['slow_k']= ta.momentum.stoch(df['High'], df['Low'], df['Close'], length, smooth)
    df['slow_d'] = ta.momentum.stoch_signal(df['High'], df['Low'], df['Close'], length, smooth)
    
# rsi
    rsi_int = 14
    df['rsi'] = ta.momentum.RSIIndicator(df['Close'], window = rsi_int).rsi()

In [10]:
def conditions(df):

    for index, row in df.iterrows():
        # c1
        df['c1'] = df['Close'] < df['bb_m']
        # c2
        df['c2'] = (df['Close'] - df['Open'] > 0) & (df['Close'].shift(1) - df['Open'].shift(1) < 0) 
        # c3
        df['c3'] = df['Volume'] > df['Volume'].shift(1)
        # c4
        df['c4'] = df['Close'] > df['Open'].shift(1)
        # c5
        df['c5'] = df['rsi'].shift(-1) >= 42

    # 條件達成
    df['signal'] = False
    df.loc[df.c1 & df.c2 & df.c3 & df.c4 & df.c5, 'signal'] = True
            
    # 下一根進場
    df['open_entry'] = False

In [11]:
def cancel_orders():
    
    open_orders = client.futures_get_open_orders()
    
    # Cancel all open orders
    if open_orders:
        # Cancel all open orders
        for order in open_orders:
            if order['positionSide'] == 'LONG':  
                cancel_response = client.futures_cancel_order(symbol=order['symbol'], orderId=order['orderId'])
                print(f"Canceled order: {order['symbol']} - {order['orderId']}")
    else:
        # Continue with the rest of the code
        print("No open LONG orders found.")
        time.sleep(1)  # Sleep for 1 second to avoid API rate limit

In [12]:
def enter_position(df):
        
    #-----Calculate entry price-----#
    close_val = df['Close']
    low_val = df['Low']
    df.loc[df.index[-1], 'entry_p'] = close_val.loc[close_val.index[-2]]
    df.loc[df.index[-1], 'stop_loss'] = low_val.loc[low_val.index[-2]]

    #-----position attributes-----#
    quantity = 0.1
    entry_p = round(df.loc[df.index[-1], 'entry_p'], 2)
    stop_loss_p = round(df.loc[df.index[-1], 'stop_loss'], 2)
    
    cancel_orders()

    try:
        order = client.futures_create_order(
            symbol=symbol.upper(),
            side='BUY',
            type='LIMIT',
            timeInForce='GTC',
            quantity=quantity,
            price=entry_p,
            positionSide='LONG',
        )
        print('Order created successfully.')       
            
    except Exception as e:
        print(f'Error creating order: {e}')
        
    print(str(symbol.upper()) + ' entered at ' + str(entry_p) + ', stop loss at ' + str(stop_loss_p))

    return df

In [13]:
def check_sl(df, current_k):
    
    # 檢查前一根 Close 是否低於 SL，若低於，即刻停損
    
    last_signal_row = df.loc[df['signal'].eq(True) & df['Close_Time'].ne(current_k)].tail(1)
    if not last_signal_row.empty:
        current_sl = round(df.loc[last_signal_row.index[0], 'Low'] * 0.99, 1)

    print(f'Current SL at {current_sl}')
    print(f'Current TP at DYNAMIC')
    
    # 確定停損
    if df.iloc[df.index[-2]]['Close'] <= current_sl:
        
        print('Last candle closed below SL, try create SL Order')
        
        cancel_orders()

        # 如果目前價格高於停損
        ticker = client.futures_symbol_ticker(symbol=symbol.upper())

        mark_price = round(float(ticker['price']), 2)
        
        stop_price = current_sl
        if current_sl > mark_price:
            stop_price = mark_price
        
        try:

            stop_loss_order = client.futures_create_order(
                symbol=symbol.upper(),
                side='SELL',
                type='STOP_MARKET',
                stopPrice=stop_price,
                stopLimitTimeInForce='GTC',
                closePosition = 'true',
                positionSide = 'LONG'
            )
        
            print(f'Stop loss created successfully at: {current_sl}')
            winsound.Beep(frequency, duration)
        
        except Exception as e:
            
            print(f'Error creating SL order: {e}')
          

In [14]:
tp_rsi = 68
tp_slow_k = 80
tp_slow_d = 80

def check_tp(df):
    global tp_rsi, tp_slow_k, tp_slow_d
    
    try:
        # 隨時偵測出場條件是否成立，不必等 Close 發生
        if ((df.loc[df.index[-1], 'rsi'] >= tp_rsi) & 
            (df.loc[df.index[-1], 'slow_k'] >= tp_slow_k) &
            (df.loc[df.index[-1], 'slow_d'] >= tp_slow_d) &
            (df.loc[df.index[-1], 'Close'] <= df.loc[df.index[-1], 'bb_u'])):
            take_profit_order = client.futures_create_order(
                symbol=symbol.upper(),
                side='SELL',
                type='TAKE_PROFIT_MARKET',
                stopPrice=df.loc[df.index[-1], 'Close'],
                stopLimitTimeInForce='GTC',
                closePosition = 'true',
                positionSide = 'LONG'
            )
            winsound.Beep(frequency, duration)
            print('Take profit created successfully')
            
        else:
            print(f"Take profit target not reached rsi:{round(df.loc[df.index[-1], 'rsi'], 2)}, k:{round(df.loc[df.index[-1], 'slow_k'], 2)}")
            
    except Exception as e:
        print(f'Error creating TP order: {e}')

In [15]:
import IPython

def restart_and_run_all():
    # Restart the kernel
    IPython.Application.instance().kernel.do_shutdown(True)

    # Re-run all code cells
    IPython.get_ipython().run_line_magic('run', '-i 000_002_LONG_bb&vol&rsi_1D.ipynb')


In [16]:
from IPython.display import clear_output

line_count = 0
max_lines = 1
current_k = 0
restart_countdown = 2 * 60 * 60 # restart in 2 hours

def console_log(df):
    global line_count, max_lines, restart_countdown, loop_end_time
    try:
        df = df.reset_index(drop=True)
        df = df.round(2)
        print('----------------------------------------------------------------------------')
        print(f"{str(df.loc[df.index[-1], 'Close_Time'])} at {str(df.loc[df.index[-1], 'Close'])}")
        print()
        print(df[['Close_Time', 'Open', 'Close', 'High', 'Low', 'Volume', 'bb_u', 'bb_m', 'rsi', 'slow_k', 'slow_d', 'c1', 'c2', 'c3', 'c4', 'c5', 'signal', 'open_entry']].tail())
        print('----------------------------------------------------------------------------')

        positions = client.futures_account()['positions']
        for position in positions:
            if float(position['positionAmt']) != 0:
                position_df = pd.DataFrame({'Symbol':position['symbol'],
                                            'Side':position['positionSide'],
                                            'Entry_P':round(float(position['entryPrice']),2),
                                            'Amt':round(float(position['positionAmt']) * df.loc[df.index[-1], 'Close'],2),
                                            'PL':round(float(position['unrealizedProfit']),2),
                                            'X':round(float(position['leverage']),1),
                                           }, index=[0])      
                print(position_df)
                print()

        # countdown for restarting session        
        restart_countdown -= 1
        print(f'{restart_countdown} iterations until restart')

        if restart_countdown <= 100:
            restart_and_run_all()
            
        # count each loop time
        loop_end_time = time.time()
        print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")            
            
    except Exception as e:
        print(f'Error UPDATING info: {e}')    

    line_count += 1
    if line_count >= max_lines:
        clear_output(wait=True)
        line_count = 0

In [17]:
def check_time(df):
    global current_k
    
    try:
        # 如果時間標籤改變，代表上個 Close 發生，檢查是否進場
        if df['Close_Time'][len(df) - 1] != current_k:

            print('time changed')
            current_k = df['Close_Time'][len(df) - 1]

            if df.iloc[df.index[-2]]['signal'] == True:

                winsound.Beep(frequency, duration)
                df.loc[df.index[-1], 'open_entry'] = True
                enter_position(df)

                print(df.tail(2))

    except Exception as e:
        print(f'Error creating ENTRY order: {e}')

In [18]:
loop_start_time = 0
loop_end_time = 0

def run():
    global current_k, restart_countdown, loop_start_time

    while True:
        loop_start_time = time.time()
        try:
            df = get_historical(symbol, interval, start_time, end_time, step)
            indicators(df)
            conditions(df)
            check_time(df)

            # 檢查停損
            try:
                positions_info = client.futures_account()['positions']
                long_positions = [p for p in positions_info if p['positionSide'] == 'LONG' and float(p['positionAmt']) != 0]

                if long_positions:
                    check_tp(df)
                    check_sl(df, current_k)

                else:
                    print('No LONG position. No SL')

            except Exception as e:
                print(f'Error checking SL, TP: {e}')

            # 更新狀態
            console_log(df)
            
            time.sleep(2)
            
        except ConnectionError as e:
            print("Connection error occurred:", e)
            print("Retrying in 5 seconds...")
            time.sleep(5)

In [ ]:
run()   

No LONG position. No SL
----------------------------------------------------------------------------
2023-05-24 07:59:59 at 1819.54

             Close_Time     Open    Close     High      Low     Volume     bb_u     bb_m    rsi  slow_k  slow_d    c1     c2     c3     c4     c5  signal  open_entry
35  2023-05-20 07:59:59  1800.56  1812.32  1829.54  1796.81  243116.47  1948.38  1857.63  40.85   25.92   25.69  True   True  False  False   True   False       False
36  2023-05-21 07:59:59  1812.31  1819.42  1828.87  1807.06  127754.43  1939.49  1853.51  42.23   41.08   29.57  True  False  False   True  False   False       False
37  2023-05-22 07:59:59  1819.42  1804.91  1828.98  1797.21  161517.81  1939.77  1852.05  40.17   44.02   37.01  True  False   True  False   True   False       False
38  2023-05-23 07:59:59  1804.91  1817.35  1828.27  1792.01  237498.94  1938.34  1850.15  42.75   52.45   45.85  True   True   True  False   True   False       False
39  2023-05-24 07:59:59  1817.36  181

In [ ]:
# c1 : Close < bb_m
# c2 : current green & previous red
# c3 : current volume > previous volume
# c4 : engulf
# c5 : rsi >= 42